<a href="https://colab.research.google.com/github/fyellow/Breastcancer_Capstone/blob/main/Resnet50_included.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RESNET-50

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Set the seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)

# Load the dataset
train_dir = 'path/to/um_bmid_gen_3/train'
val_dir = 'path/to/um_bmid_gen_3/val'
input_shape = (256, 256, 3)

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',  # set to 'categorical' for multiple classes
    seed=seed_value
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',  # set to 'categorical' for multiple classes
    seed=seed_value
)

# Create the ResNet50 model
def create_resnet_model(input_shape, num_classes):
    base_model = keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, activation='relu')(x)
    predictions = keras.layers.Dense(num_classes, activation='softmax')(x)  # set to 'softmax' for multiple classes

    model = keras.models.Model(inputs=base_model.input, outputs=predictions)
    
    return model

model = create_resnet_model(input_shape, num_classes=4)  # set to 4 for UM-BMID Gen 3 dataset

# Compile the model
model.compile(
    loss='categorical_crossentropy',  # set to 'categorical_crossentropy' for multiple classes
    optimizer=keras.optimizers.Adam(lr=1e-4),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size,
    callbacks=[
        keras.callbacks.ModelCheckpoint('resnet50_best_model.h5', save_best_only=True),
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.TensorBoard(log_dir='./logs')
    ]
)


In [ ]:

# Load the test set
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras

# Set the seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)

test_dir = 'UM_BMID_Gen_3/test'
input_shape = (224, 224, 3)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    seed=seed_value,
    shuffle=False
)

# Load the model
model = keras.models.load_model('resnet50_best_model.h5')

# Evaluate the model on the test set
scores = model.evaluate(test_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Make predictions on the test set
predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels and filenames from the generator
true_labels = test_generator.classes
filenames = test_generator.filenames

# Create a classification report
class_names = ['benign', 'malignant']
report = sklearn.metrics.classification_report(true_labels, predicted_classes, target_names=class_names)
print(report)

# Save the predicted classes and filenames
np.save('predicted_classes.npy', predicted_classes)
np.save('filenames.npy', filenames)



In [2]:
!echo "# DL_breast_cancer" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/BadrishVasu/DL_breast_cancer.git
!git push -u origin main


Initialized empty Git repository in /content/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@bf6e0780890b.(none)')
error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/BadrishVasu/DL_breast_cancer.git'
